In [122]:
import pandas as pd
import numpy as np
import torch
import os
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

In [123]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [124]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [125]:
df=pd.read_csv('RESULTS_EUROVIS2015.csv');
AUC_scores=df.tail(1);

main_df=df[:-1];
var_names=['scoreDes','fileName','indexFile', 'indexDR', 'classNum', 'scoreA', 'scoreM'];

main_df=main_df[var_names];
fileName_vals=main_df['fileName'].values;
class_vals=main_df['classNum'].values;


new_file_name_values=[]

for i in range(len(fileName_vals)):
    new_file_name_values.append(fileName_vals[i].split(".")[0]+"_cls"+str(class_vals[i])+'.jpg')
    
main_df['mod_file_name']=new_file_name_values;


mod_filename_values=main_df['mod_file_name'].values;


count=0;
# for val in mod_filename_values:
#     if val not in os.listdir('./final_imgs'):
#         count+=1;
#         print(val);

# print(count);

In [126]:
image_names=mod_filename_values;

image_labels=[[1,0] if x==0 else [0,1] for x in main_df['scoreDes'].values];
image_labels=[x for x in main_df['scoreDes'].values];
# print(image_names[:5])
# print(image_labels[:50])

In [134]:
class ScatterPlotsDataset(Dataset):
    """Scatter Plots dataset"""

    def __init__(self, image_names,image_labels,img_path,img_ext, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = image_names
        self.y_train = image_labels

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X_train[index])
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        label = torch.Tensor(np.array([self.y_train[index]]))
        return img, label
    


In [135]:
transformations = transforms.Compose([transforms.Scale(32),transforms.ToTensor()])

dset = ScatterPlotsDataset(image_names,image_labels,'/Users/yashd/PycharmProjects/CSE523/DL_Approach/final_imgs/',"",transformations)
train_loader = DataLoader(dset,batch_size=256,shuffle=True,num_workers=4)

In [136]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        
#         self.fc1 = nn.Linear(2688, 128)
#         self.fc2 = nn.Linear(128, 2)
        
#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2(x), 2))
#         x = x.view(x.size(0), -1) # Flatten layer
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.softmax(x)

# model = Net() # On CPU
# model = Net().cuda() # On GPU

In [170]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=1)
        
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        
    def forward(self, x):
        x=F.max_pool2d(self.conv1(x), 2)
        x=F.max_pool2d(self.conv2(x), 2)
        x=F.max_pool2d(self.conv3(x), 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.sigmoid(x)

model = Net() # On CPU

In [171]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)

In [174]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
#         print(data.shape)
        output = model(data)
#         print(target.shape,output.shape)
        print(output[:2])
        loss = F.binary_cross_entropy(output, target)
        
        loss.backward()
        optimizer.step()
        if batch_idx % 2 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [175]:
for epoch in range(1, 10):
    train(epoch)

Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/lo

tensor([[0.4942],
        [0.4955]], grad_fn=<SliceBackward>)
Train Epoch: 1 [0/828 (0%)]	Loss: 0.693519
tensor([[0.4949],
        [0.4949]], grad_fn=<SliceBackward>)
tensor([[0.4929],
        [0.4930]], grad_fn=<SliceBackward>)
Train Epoch: 1 [512/828 (50%)]	Loss: 0.693583
tensor([[0.4940],
        [0.4954]], grad_fn=<SliceBackward>)


Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/lo

tensor([[0.4957],
        [0.4955]], grad_fn=<SliceBackward>)
Train Epoch: 2 [0/828 (0%)]	Loss: 0.693558
tensor([[0.4986],
        [0.4990]], grad_fn=<SliceBackward>)
tensor([[0.4994],
        [0.5005]], grad_fn=<SliceBackward>)
Train Epoch: 2 [512/828 (50%)]	Loss: 0.692856
tensor([[0.4994],
        [0.5000]], grad_fn=<SliceBackward>)


Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/lo

...

tensor([[0.5010],
        [0.5015]], grad_fn=<SliceBackward>)
Train Epoch: 7 [0/828 (0%)]	Loss: 0.692762
tensor([[0.5005],
        [0.4993]], grad_fn=<SliceBackward>)
tensor([[0.4995],
        [0.4988]], grad_fn=<SliceBackward>)
Train Epoch: 7 [512/828 (50%)]	Loss: 0.692774
tensor([[0.4992],
        [0.4992]], grad_fn=<SliceBackward>)


Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/lo

tensor([[0.4991],
        [0.5000]], grad_fn=<SliceBackward>)
Train Epoch: 8 [0/828 (0%)]	Loss: 0.692785
tensor([[0.5005],
        [0.4993]], grad_fn=<SliceBackward>)
tensor([[0.4999],
        [0.4998]], grad_fn=<SliceBackward>)
Train Epoch: 8 [512/828 (50%)]	Loss: 0.692717
tensor([[0.5014],
        [0.5011]], grad_fn=<SliceBackward>)


Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _DataLoaderIter.__del__ at 0x1264db158>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/lo

tensor([[0.5013],
        [0.5018]], grad_fn=<SliceBackward>)
Train Epoch: 9 [0/828 (0%)]	Loss: 0.692770
tensor([[0.5003],
        [0.5012]], grad_fn=<SliceBackward>)
tensor([[0.5000],
        [0.4997]], grad_fn=<SliceBackward>)
Train Epoch: 9 [512/828 (50%)]	Loss: 0.692655
tensor([[0.5026],
        [0.5015]], grad_fn=<SliceBackward>)
